In [10]:
import yfinance as yf
import pandas as pd
import numpy as np
import pyfolio as pf



# List of ohlc stocks for different f&o company stocks
ohlc_stocks = ['RELIANCE.NS','HDFCBANK.NS','ICICIBANK.NS','INFY.NS','HDFC.NS','TCS.NS','ITC.NS','KOTAKBANK.NS',
                'HINDUNILVR.NS','LT.NS','SBIN.NS','BHARTIARTL.NS','BAJFINANCE.NS','AXISBANK.NS','ASIANPAINT.NS',
                'M&M.NS','MARUTI.NS','TITAN.NS','SUNPHARMA.NS','BAJAJFINSV.NS','HCLTECH.NS','ADANIENT.NS','TATASTEEL.NS',
                'INDUSINDBK.NS','NTPC.NS','POWERGRID.NS','TATAMOTORS.NS','ULTRACEMCO.NS','NESTLEIND.NS','TECHM.NS',
                'GRASIM.NS','CIPLA.NS','JSWSTEEL.NS','ADANIPORTS.NS','WIPRO.NS','HINDALCO.NS','SBILIFE.NS','DRREDDY.NS',
                'EICHERMOT.NS','HDFCLIFE.NS','ONGC.NS','TATACONSUM.NS','DIVISLAB.NS','BAJAJ-AUTO.NS','BRITANNIA.NS',
                'APOLLOHOSP.NS','COALINDIA.NS','UPL.NS','HEROMOTOCO.NS','BPCL.NS']  # List all 50 stock symbols here
  
# Define the start and end dates
start_date = "2019-01-01"
end_date = "2023-06-01"

# Create an empty DataFrame to store the historical data
df = pd.DataFrame()

# Fetch historical data for each stock
for stock in ohlc_stocks:
    stock_data = yf.download(stock, start=start_date, end=end_date)
    stock_data['Stock'] = stock
    df = df.append(stock_data)

# Reset the index
df.reset_index(inplace=True)

# Print the DataFrame
print(df)
import warnings
warnings.filterwarnings("ignore")


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [3]:
df

,Date,Open,High,Low,Close,Adj Close,Volume,Stock
0,2019-01-01,1114.683960,1116.714722,1099.676147,1110.473877,1093.406982,4498087,RELIANCE.NS
1,2019-01-02,1104.034912,1116.417480,1090.661621,1096.010864,1079.166382,7212697,RELIANCE.NS
2,2019-01-03,1097.100586,1104.133911,1079.864014,1082.489136,1065.852417,7517041,RELIANCE.NS
3,2019-01-04,1087.095459,1094.079224,1070.948486,1088.333740,1071.607300,8545382,RELIANCE.NS
4,2019-01-07,1096.605225,1107.947754,1090.661621,1094.376465,1077.557007,5565822,RELIANCE.NS
...,...,...,...,...,...,...,...,...
54545,2023-05-25,363.000000,364.200012,360.350006,363.299988,363.299988,1676285,BPCL.NS
54546,2023-05-26,365.000000,367.000000,363.100006,364.299988,364.299988,1092796,BPCL.NS
54547,2023-05-29,364.299988,367.299988,361.000000,361.700012,361.700012,1170519,BPCL.NS
54548,2023-05-30,363.000000,364.000000,360.000000,360.399994,360.399994,1361344,BPCL.NS


In [12]:
def get_top5_performers(df):
    # Calculate 52-week rolling returns for each stock
    df['RollingReturns'] = df.groupby('Stock')['Close'].transform(lambda x: x.pct_change(periods=52))

    # Get the latest date in the DataFrame
    latest_date = df['Date'].max()

    # Filter the DataFrame to include only the latest date
    latest_data = df[df['Date'] == latest_date]

    # Sort the DataFrame by rolling returns in descending order
    sorted_data = latest_data.sort_values('RollingReturns', ascending=False)

    # Get the top 5 performers
    top5_performers = sorted_data.head(5)

    return top5_performers

# Call the custom function to get the top 5 performers
top5_performers = get_top5_performers(df)

# Print the top 5 performers
print(top5_performers[['Stock', 'RollingReturns']])

               Stock  RollingReturns
24001    ADANIENT.NS        0.330212
29456  TATAMOTORS.NS        0.245975
46912    DIVISLAB.NS        0.244525
43639    HDFCLIFE.NS        0.236768
26183  INDUSINDBK.NS        0.213659


In [13]:
def calculate_atr(data):
    data['High-Low'] = data['High'] - data['Low']
    data['High-PrevClose'] = abs(data['High'] - data['Close'].shift())
    data['Low-PrevClose'] = abs(data['Low'] - data['Close'].shift())
    data['TR'] = data[['High-Low', 'High-PrevClose', 'Low-PrevClose']].max(axis=1)
    data['ATR'] = data['TR'].rolling(window=14).mean()
    return data


def go_long_top5_performers():
    # Create an empty DataFrame to store the historical data
    df = pd.DataFrame()

    # Fetch historical data for each stock
    for stock in ohlc_stocks:
        stock_data = yf.download(stock, start=start_date, end=end_date)
        stock_data['Stock'] = stock
        df = df.append(stock_data)

    # Reset the index
    df.reset_index(inplace=True)

    # Calculate rolling returns
    df['RollingReturns'] = df.groupby('Stock')['Close'].transform(lambda x: x.pct_change(52).shift(-52) * 100)

    # Get the latest date
    latest_date = df['Date'].max()

    # Filter the DataFrame to get the top 5 performers
    top5_performers = df[df['Date'] == latest_date].nlargest(5, 'RollingReturns')

    # Iterate over the top 5 performers and calculate entry price and stop loss
    for _, row in top5_performers.iterrows():
        stock = row['Stock']
        entry_price = row['Close']
        atr_data = df[df['Stock'] == stock].copy()
        atr_data = calculate_atr(atr_data)
        atr = atr_data.iloc[-1]['ATR']
        stop_loss = entry_price - (2 * atr)
        print("Go long on stock:", stock)
        print("Entry Price:", entry_price)
        print("Stop Loss:", stop_loss)
        print()


# Call the function
go_long_top5_performers()


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [6]:
import yfinance as yf
import pandas as pd


# List of top 10 stocks in the Nifty50 index
ohlc_stocks = ['RELIANCE.NS', 'HDFCBANK.NS', 'ICICIBANK.NS', 'INFY.NS', 'HDFC.NS', 'TCS.NS', 'ITC.NS', 'KOTAKBANK.NS',
               'HINDUNILVR.NS', 'LT.NS', 'SBIN.NS', 'BHARTIARTL.NS', 'BAJFINANCE.NS', 'AXISBANK.NS', 'ASIANPAINT.NS',
               'M&M.NS', 'MARUTI.NS', 'TITAN.NS', 'SUNPHARMA.NS', 'BAJAJFINSV.NS', 'HCLTECH.NS', 'ADANIENT.NS',
               'TATASTEEL.NS', 'INDUSINDBK.NS', 'NTPC.NS', 'POWERGRID.NS', 'TATAMOTORS.NS', 'ULTRACEMCO.NS',
               'NESTLEIND.NS', 'TECHM.NS', 'GRASIM.NS', 'CIPLA.NS', 'JSWSTEEL.NS', 'ADANIPORTS.NS', 'WIPRO.NS',
               'HINDALCO.NS', 'SBILIFE.NS', 'DRREDDY.NS', 'EICHERMOT.NS', 'HDFCLIFE.NS', 'ONGC.NS', 'TATACONSUM.NS',
               'DIVISLAB.NS', 'BAJAJ-AUTO.NS', 'BRITANNIA.NS', 'APOLLOHOSP.NS', 'COALINDIA.NS', 'UPL.NS',
               'HEROMOTOCO.NS', 'BPCL.NS']

# Define the start and end dates
start_date = "2019-01-01"
end_date = "2023-06-01"


def calculate_atr(data):
    data['High-Low'] = data['High'] - data['Low']
    data['High-PrevClose'] = abs(data['High'] - data['Close'].shift())
    data['Low-PrevClose'] = abs(data['Low'] - data['Close'].shift())
    data['TR'] = data[['High-Low', 'High-PrevClose', 'Low-PrevClose']].max(axis=1)
    data['ATR'] = data['TR'].rolling(window=14).mean()
    return data


def rebalance_stocks():
    # Create an empty DataFrame to store the historical data
    df = pd.DataFrame()

    # Fetch historical data for each stock
    for stock in ohlc_stocks:
        stock_data = yf.download(stock, start=start_date, end=end_date)
        stock_data['Stock'] = stock
        df = df.append(stock_data)

    # Reset the index
    df.reset_index(inplace=True)

    # Calculate rolling returns
    df['RollingReturns'] = df.groupby('Stock')['Close'].transform(lambda x: x.pct_change(52).shift(-52) * 100)

    # Get the latest date
    latest_date = df['Date'].max()

    # Filter the DataFrame to get the top 10 performers
    top10_performers = df[df['Date'] == latest_date].nlargest(10, 'RollingReturns')

    # Get the list of stocks for rebalancing
    rebalanced_stocks = top10_performers['Stock'].tolist()

    # Print the rebalanced stocks
    print("Rebalanced Stocks:")
    for stock in rebalanced_stocks:
        print(stock)

    # Print an empty line
    print()


# Call the function to rebalance the stocks
rebalance_stocks()


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [20]:
import yfinance as yf
import pandas as pd
import pyfolio as pf
import warnings
warnings.filterwarnings("ignore")

# List of ohlc stocks for different F&O company stocks
ohlc_stocks = ['RELIANCE.NS','HDFCBANK.NS','ICICIBANK.NS','INFY.NS','HDFC.NS','TCS.NS','ITC.NS','KOTAKBANK.NS',
                'HINDUNILVR.NS','LT.NS','SBIN.NS','BHARTIARTL.NS','BAJFINANCE.NS','AXISBANK.NS','ASIANPAINT.NS',
                'M&M.NS','MARUTI.NS','TITAN.NS','SUNPHARMA.NS','BAJAJFINSV.NS','HCLTECH.NS','ADANIENT.NS','TATASTEEL.NS',
                'INDUSINDBK.NS','NTPC.NS','POWERGRID.NS','TATAMOTORS.NS','ULTRACEMCO.NS','NESTLEIND.NS','TECHM.NS',
                'GRASIM.NS','CIPLA.NS','JSWSTEEL.NS','ADANIPORTS.NS','WIPRO.NS','HINDALCO.NS','SBILIFE.NS','DRREDDY.NS',
                'EICHERMOT.NS','HDFCLIFE.NS','ONGC.NS','TATACONSUM.NS','DIVISLAB.NS','BAJAJ-AUTO.NS','BRITANNIA.NS',
                'APOLLOHOSP.NS','COALINDIA.NS','UPL.NS','HEROMOTOCO.NS','BPCL.NS']  # List all 50 stock symbols here

# Define the start and end dates
start_date = "2019-01-01"
end_date = "2023-06-01"

# Create an empty DataFrame to store the historical data
df = pd.DataFrame()

# Fetch historical data for each stock
for stock in ohlc_stocks:
    stock_data = yf.download(stock, start=start_date, end=end_date)
    stock_data['Stock'] = stock
    df = df.append(stock_data)

# Reset the index
df.reset_index(inplace=True)

# Define the custom function to calculate ATR
def calculate_atr(data):
    data['High-Low'] = data['High'] - data['Low']
    data['High-PrevClose'] = abs(data['High'] - data['Close'].shift())
    data['Low-PrevClose'] = abs(data['Low'] - data['Close'].shift())
    data['TR'] = data[['High-Low', 'High-PrevClose', 'Low-PrevClose']].max(axis=1)
    data['ATR'] = data['TR'].rolling(window=14).mean()
    return data

# Define the custom function to generate trading signals and calculate returns
def generate_signals(data):
    signals = pd.DataFrame(index=data.index)
    signals['Signal'] = 0

    for stock in ohlc_stocks:
        atr_data = data[data['Stock'] == stock].copy()
        atr_data = calculate_atr(atr_data)

        atr = atr_data['ATR'].iloc[-1]
        atr_multiplier = 2

        signals.loc[(data['Stock'] == stock) & (data['Close'] > data['Close'].shift()) & (
                    data['Close'] > (data['Close'].shift() + atr_multiplier * atr)), 'Signal'] = 1

        signals.loc[(data['Stock'] == stock) & (data['Close'] < data['Close'].shift()) & (
                    data['Close'] < (data['Close'].shift() - atr_multiplier * atr)), 'Signal'] = -1

    return signals

# Generate trading signals
signals = generate_signals(df)

# Calculate the daily returns
df['Return'] = df.groupby('Stock')['Close'].pct_change()

# Apply the trading signals to calculate strategy returns
df['StrategyReturn'] = df['Return'] * signals['Signal'].shift()

# Drop NaN values from the DataFrame
df.dropna(inplace=True)

# Calculate the cumulative returns
df['CumulativeReturn'] = (1 + df['StrategyReturn']).cumprod()

# Print the DataFrame
print(df)

# Perform the backtest analysis using pyfolio
returns = df[['Date', 'CumulativeReturn']]
returns.set_index('Date', inplace=True)

# Calculate additional performance metrics using pyfolio
returns = returns.pct_change().fillna(0)
returns.columns = ['Strategy']

# Fetch benchmark data and align the index
benchmark = yf.download('^NSEI', start=start_date, end=end_date)['Close'].pct_change().fillna(0)
benchmark = benchmark.reindex(returns.index)
benchmark.columns = ['Benchmark']

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [24]:
pd.set_option('display.max_rows', None)
df.tail()

,Date,Open,High,Low,Close,Adj Close,Volume,Stock,Return,StrategyReturn,CumulativeReturn
54545,2023-05-25,363.000000,364.200012,360.350006,363.299988,363.299988,1676285,BPCL.NS,0.000000,0.0,0.014629
54546,2023-05-26,365.000000,367.000000,363.100006,364.299988,364.299988,1092796,BPCL.NS,0.002753,0.0,0.014629
54547,2023-05-29,364.299988,367.299988,361.000000,361.700012,361.700012,1170519,BPCL.NS,-0.007137,-0.0,0.014629
54548,2023-05-30,363.000000,364.000000,360.000000,360.399994,360.399994,1361344,BPCL.NS,-0.003594,-0.0,0.014629
54549,2023-05-31,360.399994,366.450012,360.200012,363.500000,363.500000,11607240,BPCL.NS,0.008602,0.0,0.014629
